### riss 저널 크롤링 코드

In [47]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import matplotlib.pyplot as plt

In [48]:
# 논문 키워드와 편수를 입력하면 soup객체를 return하는 함수 정의
def get_soup(keyword, num=100):

    HEADERS= {'User-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

    url = f'https://riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query={keyword}\
&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale={num}\
&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery={keyword}&\
fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword={keyword}\
&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&db_type=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn='
    result = requests.get(url, headers=HEADERS)

    if result.status_code == 200:
        print("OK")
        soup = bs(result.content, 'html.parser') #result.text

    return soup


In [49]:
# 함수 호출
soup = get_soup('한국운동역학회', 2121)


OK


#### riss 저널의 "한국운동역학회" 제목, 연도, 초록 크롤링

In [59]:
# soup를 넣으면 판다스 df를 return하는 함수 정의
def get_riss_df(soup):

    contents = soup.find_all('div', class_= 'cont ml60')

    title = []
    year = []
    journal = []
    abstracts = []

    for cont in contents :
        title.append(cont.find('p', class_='title').text)
        year.append(cont.find('p', class_='etc').find_all('span')[2].text)
        journal.append(cont.find('p', class_='etc').find_all('span')[3].text)
        abstracts.append(cont.find('p', class_='preAbstract').text)

        if cont.find('p', class_='preAbstract'):
            abstracts.append(cont.find('p', class_='preAbstract').text)
        else :
            abstracts.append('초록이 없습니다.')

    df = pd.DataFrame(
        {'title' : title,
         'year' : year,
         'journal': journal,
         'abstracts': abstracts})

    return df

In [60]:
df = get_riss_df(soup)
df

,title,year,journal,abstracts
0,스포츠산업과 운동역학 : 한국운동역학회 현황과 과제(1),2005,한국운동역학회 추계학술발표논문집,초록이 없습니다.
1,한국운동역학회의 야긴과 보아스!,2009,한국운동역학회지,초록이 없습니다.
2,한국운동역학회 현황과 발전방향 과제(1),2006,한국운동역학회지,초록이 없습니다.
3,한국운동역학의 초석이신 선배님께,2009,한국운동역학회지,초록이 없습니다.
4,한국운동역학회지의 국제화와 SCI-E 등재 방안,2010,한국운동역학회지,Korean Journal of Sport Biomechanics(KJSB) is ...
...,...,...,...,...
2116,Gait Pattern of Hemiplegic Patients with Swimm...,2019,한국운동역학회지,Objective: The purpose of this study was to in...
2117,Effect of Wearing Ankle Weights on Underwater ...,2019,한국운동역학회지,Objective: The main purpose of this study was ...
2118,Age-related Changes in Multi-finger Synergy du...,2016,한국운동역학회지,Objective: The aim of this study was to invest...
2119,초등체육에서 운동역학의 현실과 연구 활성화 방안,2016,한국초등체육학회지,체육학에서 운동역학 및 초등체육 관련 운동역학 연구의 현실과 문제점을 알아보고 활성...


In [61]:
df['journal'].value_counts()

journal
한국운동역학회지             1657
한국운동역학회 논문 영문초록집      370
한국운동역학회 춘계학술발표논문집      60
한국운동역학회 추계학술발표논문집      32
한국초등체육학회지               1
대한태권도과학연구소 논문집          1
Name: count, dtype: int64

* 한국운동역학회가 아닌 한국초등체육학회지 / 대한태권도과학연구소 논문집은 삭제

In [62]:
df[df['journal'] == "한국초등체육학회지"]

,title,year,journal,abstracts
2119,초등체육에서 운동역학의 현실과 연구 활성화 방안,2016,한국초등체육학회지,체육학에서 운동역학 및 초등체육 관련 운동역학 연구의 현실과 문제점을 알아보고 활성...


In [63]:
df[df['journal'] == "대한태권도과학연구소 논문집"]

,title,year,journal,abstracts
2120,국내 합기도 연구 동향과 향후 연구방향에 대한 고찰 : 태권도의 사례를 중심으로,2013,대한태권도과학연구소 논문집,본연구는 국내 합기도의 연구동향을 분석하고 태권도의 사례로 합기도의 발전방향을 제시...


In [64]:
df = df.drop([2119,2120])

In [65]:
df['journal'].value_counts()

journal
한국운동역학회지             1657
한국운동역학회 논문 영문초록집      370
한국운동역학회 춘계학술발표논문집      60
한국운동역학회 추계학술발표논문집      32
Name: count, dtype: int64

* 제목의 중복을 제거

In [66]:
df[df.duplicated(keep=False)]['title'].value_counts()

title
Qualitative Analysis of Pressure Intensity and Center of Pressure Trajectory According to Shoe Type                                                      3
Effects of Rhythmic Hop on Response Times and Kicking Velocities of Taekwondo Kicks                                                                      3
The Effects of Shoe Type on Ground Reaction Force                                                                                                        3
Effects of Water Exercise on the Foot Pressure Distribution of a Female Adult with Hemiplegia: A Biomechanical Case Study                                3
The Immediate Effects of Five-Toed Shoes on Foot Structure                                                                                               2
Validity of a Portable APDM Inertial Sensor System for Stride Time and Stride Length during Treadmill Walking                                            2
노인 보행 시 하지 근 활동 양상과 관절의 안정성이 낙상에 미치는 영향 -전향적 연구(Prospective Stud

In [67]:
df = df.drop_duplicates(['title'])

* 초록의 중복을 제거

In [72]:
df['abstracts'].value_counts()

abstracts
초록이 없습니다.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [77]:
df = df.drop_duplicates(['abstracts'])

* 초록이 없는 데이터 제거

In [78]:
df = df[df['abstracts'] != '초록이 없습니다.']

In [79]:
df['title'].value_counts()

title
한국운동역학회지의 국제화와 SCI-E 등재 방안                                                                                                           1
Acute Changes in Fascicle Behavior and Electromyographic Activity of the Medial Gastrocnemius during Walking in High Heeled Shoes    1
Effect of Prolonged Running-induced Fatigue on Free-torque Components                                                                1
Comparison of Biomechanical Characteristics of Rowing Performance between Elite and Non-Elite Scull Rowers: A Pilot Study            1
The Immediate Effects of External Kinesio-tape Wrapping for Inner Arch Support on the Lower Leg EMG for Gait in Stiletto Heels       1
                                                                                                                                    ..
손 끝 온도변화가 젊은 성인의 다중 손가락 동작에 미치는 효과                                                                                                   1
런닝화의 미드솔 경도가 하지 근육의 피로와 충격력에 미치는 영향              

In [80]:
df['abstracts'].value_counts()

abstracts
Korean Journal of Sport Biomechanics(KJSB) is facing down by a great challenge. Following the long-term plan of the Korean Society of Sport Biomechanics, the KJSB has entered a new era in 2010 in the sense that the some papers are published in English. Immense efforts are needed to accomplish the original goal of KJSB becoming an internationally recognized journal. The 2009 Spring International Conference of the KSSB was held in this connection with the subject of the vitalization of the KJSB. A special topic were presented at the Conference and vigorous discussions followed. The Editor-in-Chief of KJSB presented strategies to improve the number of suggestion to the KJSB. The Manager of Thomson Reuter gave a presentation on how KJSB can grow into the SCI-expanded journal. This manuscript is presented on the basis of the presentations and the discussions at the meeting for the purpose of finding the way in aiming the direction of the long-term goal of the KJSB. Other evaluatio

In [81]:
data2['year'].value_counts()

year
2016    147
2017     99
2011     82
2019     77
2018     68
2010     59
2020     58
2014     55
2015     54
2001     51
2012     49
2004     48
2013     48
2003     47
2002     43
2021     41
2000     37
2023     21
2022     20
2005     18
1998     15
1997     11
1999     10
1996      7
2009      6
2006      6
1994      5
2024      5
1995      2
2008      1
Name: count, dtype: int64

In [82]:
df

,title,year,journal,abstracts
4,한국운동역학회지의 국제화와 SCI-E 등재 방안,2010,한국운동역학회지,Korean Journal of Sport Biomechanics(KJSB) is ...
5,스쿼트 동작 시 경사기능전신진동기의 적용이 몸통 및 하지 근 활성도에 미치는 영향,2015,한국운동역학회지,Objective : The purpose of this study was to i...
6,한국형 봅슬레이화 개발을 위한 구간 시간과 족저압력 분석,2015,한국운동역학회지,Objective : A study and development of Korean ...
7,보행 시 농작업 종사자들의 슬관절 퇴행성 등급에 따른 지면반력 특성 분석,2015,한국운동역학회지,Objective : This study was conducted with an a...
8,편평족 중학생의 고관절 교정 운동 프로그램과 발교정구 착용 유무가 안정시 종골기립 ...,2015,한국운동역학회지,Objective : The purpose of this study is to ev...
...,...,...,...,...
1966,여중생의 체지방율에 따른 보행 특성 분석,2001,한국운동역학회지,This study is purpose to present the specifica...
1984,Effects of shoe and landing heights on impact ...,2003,한국운동역학회지,본 연구의 목적은 뛰어 내리기 동작 시 신발과 뛰어 내리는 높이가 지면반력과 충격감...
1990,REVIEW OF COMPUTATIONAL MODELS FOR FOOTWEAR DE...,2009,한국운동역학회지,Existing footwear biomechanics studies rely on...
2060,Aging-Related Changes in Hand Intrinsic and Ex...,2015,한국운동역학회지,Object : The purpose of this study was to inve...
